# Processando entradas do dataframe de métricas

## Importando os módulos necessários

In [ ]:
%pip install openpyxl

from tensorflow import keras
import tensorflow_hub as hub
import pandas as pd

## Carregando modelo já criado em "model-making-script"

In [ ]:
searchModel = keras.models.load_model(
    '../model-making-scripts/searchModel.h5', custom_objects={'KerasLayer': hub.KerasLayer})

## Carregando os dados processados em json

In [ ]:
import json

data_file_path = './data/data.json'

def getCoursesData():

    with open(data_file_path) as json_file:
        data = json.load(json_file)
    
    return data

coursesData = getCoursesData()

## Carregando o dataframe de métrica e acessando as entradas.
+ O dataframe de métrica possui uma coluna com uma pesquisa usada para buscar um artigo no espaço vetorial
+ Possui também mais duas colunas, um artigo esperado e um indice para comparação
+ A coluna artigo retornado e saída, são para a comparação dos dados de entrada e saída, caso o artigo retornado seja igual ao esperado, a saída é 1, caso contrário é 0.

In [ ]:
df = pd.read_excel('./data/Dataframe para Métrica.xlsx')

entradas = df['entrada (português)']

df

## Processando as entradas e atribuindo os resultados na coluna "curso retornado"
+ O resultado das buscas é feito com base na função de similaridade do cosseno

In [ ]:
import numpy as np
from scipy.stats import pearsonr

retornos = []

for entrada in entradas:
    
    entrada_str = str(entrada)

    retorno = searchModel.predict([entrada_str])  # Faça a previsão com base nas entradas
    retorno = np.array(retorno)  # Converte para um array numpy
    similarity_scores = []
    for course_data in coursesData["data"]:
        course_data = np.array(course_data)  # Converte para um array numpy
        similarity_scores.append(pearsonr(retorno.flatten(), course_data.flatten())[0])
    indexes = np.argsort(similarity_scores)[::-1]  # Ordenar em ordem decrescente
    retornos.append(coursesData['num'][indexes[0]])

    
df['artigo retornado'] = retornos

df


In [ ]:
# !pip install unidecode
# from unidecode import unidecode

# ignore_list = ['the', 'o', 'a']

# def parse_name(full_name):
#     name_list = full_name.split() # Separa cada nome
#     new_name_list = []
#     for name in name_list: # Percorre cada nome
#         name = name.strip('.') # Remove pontos
#         name = name.lower() # Converte todas as letras em minúsculas
#         if name in ignore_list: # Remove preposições
#             continue
        # name = unidecode(name.decode('utf8')) # Remove acentos (necessita da biblioteca 'unidecode')
#         new_name_list.append(name)
#     return new_name_list

# def is_similar(a, b):
#     a = parse_name(a)
#     b = parse_name(b)
#     if len(a) != len(b): # Se o número de palavras for diferente, retorna falso
#         return False
#     for x, y in zip(a, b):
#         if (len(x) == 1) or (len(y) == 1): # Se uma das palavras possuir apenas uma letra...
#             if x[0] != y[0]: #...compara apenas a primeira letra
#                 return False
#         else: # Caso contrário...
#             if x != y: #...compara a palavra toda
#                 return False
#     return True # Se todas as palavras forem iguais, retorna verdadeiro

## Comparando o retorno e gerando uma pontuação na saída

In [ ]:
df['saida'] = 0

for i in range(len(df['artigo esperado'])):
    if df['artigo esperado'][i] == df['artigo retornado'][i]:
        df['saida'][i] = 1

df

In [ ]:
df.head(60)

## Pontuação total da métrica

In [ ]:
score = sum(df['saida'])

score